In [1]:
import sys 
sys.path.append("../..")
sys.path.append("..")

import os
from functools import partial
from importlib import reload
from tqdm import tqdm

import torch 
from sae_training.utils import LMSparseAutoencoderSessionloader
from huggingface_hub import hf_hub_download
import plotly_express as px 
import pandas as pd 

# TORCH DON'T TRACK GRADS
torch.set_grad_enabled(False)

from transformer_lens import HookedTransformer

# model = HookedTransformer.from_pretrained("gpt2-small")

In [2]:
import wandb
run = wandb.init()
artifact = run.use_artifact('jbloom/mats_sae_training_gpt2_ghost_grad_experiment/sparse_autoencoder_gpt2-small_blocks.3.hook_resid_pre_12288:v18', type='model')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jbloom. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact sparse_autoencoder_gpt2-small_blocks.3.hook_resid_pre_12288:v18, 72.10MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.5


In [6]:
path_to_sae = "artifacts/sparse_autoencoder_gpt2-small_blocks.3.hook_resid_pre_12288:v18/270000128_sparse_autoencoder_gpt2-small_blocks.3.hook_resid_pre_12288.pt"

model, sparse_autoencoder, activation_store = LMSparseAutoencoderSessionloader.load_session_from_pretrained(path_to_sae)

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

In [7]:
from sae_training.evals import run_evals
sparse_autoencoder.eval()
metrics = run_evals(sparse_autoencoder, activation_store, model, 0)

In [8]:
metrics

{'metrics/l2_norm': 25.5283203125,
 'metrics/l2_ratio': 0.9211738109588623,
 'metrics/CE_loss_score': tensor(0.7211, device='cuda:0'),
 'metrics/ce_loss_without_sae': tensor(3.3924, device='cuda:0'),
 'metrics/ce_loss_with_sae': tensor(6.3396, device='cuda:0'),
 'metrics/ce_loss_with_ablation': tensor(13.9600, device='cuda:0')}

In [12]:
from transformer_lens.utils import get_act_name
from sae_training.evals import get_recons_loss
import numpy as np

hook_point = sparse_autoencoder.cfg.hook_point
hook_point_layer = sparse_autoencoder.cfg.hook_point_layer
hook_point_head_index = sparse_autoencoder.cfg.hook_point_head_index

### Evals
eval_tokens = activation_store.get_batch_tokens()

# Get Reconstruction Score
recons_score, ntp_loss, recons_loss, zero_abl_loss = get_recons_loss(
    sparse_autoencoder, model, activation_store, eval_tokens
)

# get cache
_, cache = model.run_with_cache(
    eval_tokens,
    prepend_bos=False,
    names_filter=[get_act_name("pattern", hook_point_layer), hook_point],
)

# get act
if sparse_autoencoder.cfg.hook_point_head_index is not None:
    original_act = cache[sparse_autoencoder.cfg.hook_point][
        :, :, sparse_autoencoder.cfg.hook_point_head_index
    ]
else:
    original_act = cache[sparse_autoencoder.cfg.hook_point]

sae_out, feature_acts, _, _, _, _ = sparse_autoencoder(original_act)
patterns_original = (
    cache[get_act_name("pattern", hook_point_layer)][:, hook_point_head_index]
    .detach()
    .cpu()
)
del cache

if "cuda" in str(model.cfg.device):
    torch.cuda.empty_cache()

if sparse_autoencoder.cfg.normalize_activations:
    l2_norm_in = np.sqrt(sparse_autoencoder.cfg.d_in)
else:
    l2_norm_in = torch.norm(original_act, dim=-1)
l2_norm_out = torch.norm(sae_out, dim=-1)
l2_norm_ratio = l2_norm_out / l2_norm_in